In [ ]:
# !pip install patool
!pip install mediapipe
# !pip install scikit-learn


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 31.1 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
# import pandas as pd
from matplotlib import pyplot as plt
import cv2

import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization, ReLU, Dropout, GRU, Bidirectional, Conv2D,Reshape
#, ConvLSTM2D, Conv3D, Flatten
from tensorflow.keras.utils import to_categorical
import random 
from sklearn.model_selection import train_test_split
import mediapipe as mp
import pickle as pk
# import glob

# import patoolib
# //from pyunpack import Archive

from google.colab import drive
from google.colab import files

In [ ]:
drive.mount("/content/drive", force_remount=True)
%cd drive/MyDrive/Colab Notebooks/

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks


In [ ]:
# patoolib.extract_archive("TrainData.rar", outdir="./traindata")
# Archive('a.zip').extractall('/path/to')

In [ ]:
path ="./traindata/TrainData/"
file_list = os.listdir(path)

files = file_list[0:10]
words = files

In [ ]:
np.unique(words)

array(['morning', 'my', 'na', 'name', 'nice', 'no', 'not', 'number',
       'okay', 'picture'], dtype='<U7')

In [ ]:
formLableToNum = {label:num for num, label in enumerate(words)}
formNumTOLabel = {num:label for num, label in enumerate(words)}
formLableToNum

{'morning': 0,
 'my': 1,
 'na': 2,
 'name': 3,
 'nice': 4,
 'no': 5,
 'not': 6,
 'number': 7,
 'okay': 8,
 'picture': 9}

In [ ]:
vids=[]
labels=[]
vidsnum = 0
for w in words:
    vidpath = path+w
    vidlist = os.listdir(vidpath)
    print(w,str(len(vidlist)))
    for i in vidlist:
        vidsnum+=1
        vid = []
        labels.append(formLableToNum[w])
        for frame_num in range(20):
            arr = np.load(vidpath+"/"+i+"/"+str(frame_num)+".npy")
            vid.append(arr)
        vid = np.reshape(np.array(vid),(20,258))
        m = vid.mean(axis=1)
        vid = vid.T
        vid -= m
        vids.append(vid)
        
    
labels = np.reshape(np.array(labels),(-1,1))

morning 81


KeyboardInterrupt: ignored

In [ ]:
vidsnum

In [ ]:
vids = np.reshape(np.array(vids),(vidsnum,20,258))

In [ ]:
vids.shape

In [ ]:
X_train, X_dev, y_train, y_dev = train_test_split(vids, labels, test_size = 0.2,shuffle=True,random_state=16)



In [ ]:
X_train.shape,y_train.shape

In [ ]:
y_train[0:20]

In [ ]:
del vids
del labels

model_lstm_1 = Sequential([

tf.keras.Input(shape=(20,258),),
    
    
# GRU(64, return_sequences=True, activation='tanh', recurrent_dropout=0),
# Dropout(0.2, noise_shape =  (1,20, 1)),

Bidirectional(GRU(32, return_sequences=True, activation='relu', recurrent_dropout=0.2),input_shape=(20,258)),
Dropout(0.4, noise_shape =  (1,20, 1)),
       
GRU(64, return_sequences=False, activation='relu', recurrent_dropout=0.2),#,recurrent_dropout=0
BatchNormalization(),
Dropout(0.4,),

Dense(64, activation='linear',kernel_regularizer=tf.keras.regularizers.L1L2(0.5,8)),#kernel_regularizer=tf.keras.regularizers.L2(0)
BatchNormalization(),#axis=-1,center=True,scale=True,
ReLU(), 


Dense(32, activation='linear',kernel_regularizer=tf.keras.regularizers.L1L2(1,16)),#kernel_regularizer=tf.keras.regularizers.L2(0)
BatchNormalization(),#axis=-1,center=True,scale=True,
ReLU(),    

Dense(25, activation='softmax')
])

In [ ]:
model_lstm_1 = Sequential([

tf.keras.Input(shape=(20,258),),

# Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"),   
# Conv2D(filters=1, kernel_size=1, padding="same", activation="relu"),
# Reshape((20,258)),



# Bidirectional(LSTM(32, return_sequences=True, activation='tanh'),input_shape=(20,258)),## Dropout(0.2, noise_shape =  (1,20, 1)),#,recurrent_dropout=0
# Dropout(0.2, noise_shape =  (1,20, 1)),

LSTM(64, return_sequences=True, activation='tanh'),
# Dropout(0.2, noise_shape =  (1,20, 1)),

LSTM(64, return_sequences=True, activation='tanh'),
# Dropout(0.2, noise_shape =  (1,20, 1)),

LSTM(128, return_sequences=False, activation='tanh'),
BatchNormalization(),#axis=-1,center=True,scale=True,
Dropout(0.5),


Dense(128, activation='linear',kernel_regularizer=tf.keras.regularizers.L1L2(0.01,2)),
BatchNormalization(),#axis=-1,center=True,scale=True,
ReLU(),

Dense(64, activation='linear',kernel_regularizer=tf.keras.regularizers.L1L2(0.01,2)),  
BatchNormalization(),#axis=-1,center=True,scale=True,
ReLU(), 

Dense(64, activation='linear',kernel_regularizer=tf.keras.regularizers.L1L2(0.05,4)),  
BatchNormalization(),#axis=-1,center=True,scale=True,
ReLU(), 
# Dropout(0.2,),

Dense(11, activation='softmax')
])


In [ ]:
model_lstm_1.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 20, 64)            82688     
                                                                 
 lstm_10 (LSTM)              (None, 20, 64)            33024     
                                                                 
 lstm_11 (LSTM)              (None, 128)               98816     
                                                                 
 batch_normalization_3 (Batc  (None, 128)              512       
 hNormalization)                                                 
                                                                 
 dense_12 (Dense)            (None, 128)               16512     
                                                                 
 dense_13 (Dense)            (None, 64)                8256      
                                                      

In [ ]:
model_lstm_1.compile(tf.keras.optimizers.Adam(learning_rate=1e-3,beta_1=0.9,beta_2=0.999,epsilon=1e-07,)
        ,loss=tf.keras.losses.SparseCategoricalCrossentropy()
            ,metrics=["accuracy"],)#

In [ ]:
model_lstm_1.fit(X_train, y_train, epochs=512, batch_size=128)#batch_size=64, validation_data=(X_dev, y_dev)

Epoch 1/512
5/5 [==============================] - 6s 16ms/step - loss: 468.4728 - accuracy: 0.2081
Epoch 2/512
5/5 [==============================] - 0s 17ms/step - loss: 434.1873 - accuracy: 0.3372
Epoch 3/512
5/5 [==============================] - 0s 17ms/step - loss: 401.8560 - accuracy: 0.3758
Epoch 4/512
5/5 [==============================] - 0s 15ms/step - loss: 371.3926 - accuracy: 0.3893
Epoch 5/512
5/5 [==============================] - 0s 16ms/step - loss: 342.8027 - accuracy: 0.4161
Epoch 6/512
5/5 [==============================] - 0s 16ms/step - loss: 316.0033 - accuracy: 0.4748
Epoch 7/512
5/5 [==============================] - 0s 16ms/step - loss: 290.9420 - accuracy: 0.4883
Epoch 8/512
5/5 [==============================] - 0s 16ms/step - loss: 267.5699 - accuracy: 0.5470
Epoch 9/512
5/5 [==============================] - 0s 16ms/step - loss: 245.8049 - accuracy: 0.5436
Epoch 10/512
5/5 [==============================] - 0s 15ms/step - loss: 225.5514 - accuracy: 0.5671

In [ ]:
model_lstm_1.evaluate(X_train, y_train)

***

In [ ]:
model_lstm_1.evaluate(X_dev, y_dev)

In [ ]:
model_lstm_1.save("./modelAsl/v8")

****
****

In [ ]:
reconstructed_model = tf.keras.models.load_model("./modelAsl/v8")

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return  results

In [ ]:
def extract_keypoints(results):

    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

In [ ]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [ ]:
holistic = mp_holistic.Holistic(min_detection_confidence=0.25, min_tracking_confidence=0.25) 

In [ ]:
def videoProc(path, c=10,c2=0):
    "###"
    cap = cv2.VideoCapture(path)
    success, frame = cap.read()
    
    fno = 0
    framecount = 20

    
    v = []
    
    skipCounter = c
    fpsCounter = c2

    while True and success:
        success, frame = cap.read()
        if fno >=framecount:
            break 
        
     
        
        if skipCounter==0:
            pass
        else:
            skipCounter-=1
            continue
        
        if fpsCounter==0:
            fpsCounter=c2
        else:
            fpsCounter-=1
            continue

        if success:
            frame = cv2.resize(frame, (720,480), interpolation = cv2.INTER_AREA)
            results = mediapipe_detection(frame, holistic)
            eres = extract_keypoints(results)

            tmp=  np.reshape(eres, (1,258))
            v.append(tmp)
            
            # draw_styled_landmarks(frame, results)
            # plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            # plt.show()
            fno += 1
        else:
            break

# if cv2.waitKey(1) & 0xFF == ord('q') or ret==False :
    cap.release()
    cv2.destroyAllWindows()
    
    f = len(v)
    v = np.array(v) if f >0 else[]
    v =np.reshape(v,(f,225))if f >0 else[]
   


    if f>0:      
        m = v.mean(axis=-1)     
        tmpv = v.T 
        tmpv -=  m
        v = tmpv.T
        m = v.mean(axis=0) 
        v -= m
    if fno<framecount and fno>0:
        tmp = np.zeros((framecount-fno,225))
        v = np.concatenate((v, tmp), axis=0) 

    return v

In [ ]:
v = videoProc("./TestDataDD/z1.mp4",20)
# 1 0 3 2 4


(20) 16 (9) 7 4

In [ ]:
v[17].min(),v[17].max()

In [ ]:
p = reconstructed_model.predict_step(np.reshape(v, (1,20,258)))

In [ ]:
np.argmax(p),p

In [ ]:
reconstructed_model.summary()